In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

wiki,arxiv

(WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Asthir Kushwaha\\Documents\\Projects\\projects\\Projects\\langchain-projects\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)))

In [2]:
tools=[wiki,arxiv]

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\Asthir Kushwaha\Documents\Projects\projects\Projects\langchain-projects\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
import os 
from dotenv import load_dotenv

load_dotenv()

os.environ['HUGGING_FACE'] = os.getenv('HUGGING_FACE')

embeddings = HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

In [7]:
loader = WebBaseLoader('https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2').load()
document = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100).split_documents(loader)
retriever = FAISS.from_documents(document,embeddings).as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000015044372F30>, search_kwargs={})

In [13]:
from langchain_core.tools import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,'huggingface-search','Search any information about the model')
retriever_tool

StructuredTool(name='huggingface-search', description='Search any information about the model', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x0000015072AF1300>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x0000015072AF0CC0>)

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [15]:
os.environ['GROQ_API_KEY']= os.getenv('GROQ_API_KEY')

llm = ChatGroq(model='llama-3.3-70b-versatile')

In [29]:
# prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assistant'),
        MessagesPlaceholder('chat_history',optional=True),
        ('human','{input}'),
        MessagesPlaceholder('agent_scratchpad',optional=True),
    ]
)

agent = create_tool_calling_agent(llm,tools,prompt)
agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad', 'chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [31]:
agent_executor.invoke({'input':'can you fetch any research paper and explain its pros over other models'})




> Entering new AgentExecutor chain...

Invoking: `huggingface-search` with `{'query': 'research paper on transformer models'}`


sentence-transformers/all-MiniLM-L6-v2 · Hugging Face









Hugging Face







						Models

						Datasets

						Spaces



			Community
		


						Docs

						Enterprise

Pricing
			






Log In
				
Sign Up
					








sentence-transformers

/
all-MiniLM-L6-v2



like
4.36k


Follow

Sentence Transformers
3.41k




Sentence Similarity


sentence-transformers


PyTorch

google-tensorflow
TensorFlow


Rust


ONNX


Safetensors


OpenVINO


Transformers



21 datasets



English


bert


feature-extraction



text-embeddings-inference



arxiv:
5 papers




License:
apache-2.0





	Model card
	

	
		
Files
Files and versions

xet



	Community
	146











			Deploy
		
















			Use this model
		











all-MiniLM-L6-v2
Usage (Sentence-Transformers)

Usage (HuggingFace Transformers)

Background

Intended uses

Training procedure
Pre-

{'input': 'can you fetch any research paper and explain its pros over other models',
 'output': 'The research paper discusses the use of transformer models in natural language processing, specifically in the context of clinical research using electronic health records (EHRs). The paper presents a framework for developing and applying transformer models to EHR-based clinical research, using the National COVID Cohort Collaborative (N3C) as a case study.\n\nThe paper highlights the potential of transformer models to improve the accuracy and efficiency of clinical research, particularly in the context of large-scale datasets like EHRs. The authors demonstrate the effectiveness of their framework by applying it to a variety of tasks, including named entity recognition, sentiment analysis, and question answering.\n\nThe pros of using transformer models in this context include:\n\n1. Improved accuracy: Transformer models have been shown to achieve state-of-the-art results in a variety of natu